In [1]:
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=47cb0776eb36c6ebd944034dc11a76af1be9eb023dfa7fc827e68aafccf980c2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-fqyhqrio
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-fqyhqrio
  Resolved https://github.com/huggingface/transformers to commit 4f2bf135aff70bcb231fba1ad6723564ec6103d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 31.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9967353 sha256=790f8c675682aed8ea6fb20ef0951524f8d50579c16d226ff41f9bfeb7aa1af1
  Stored in directory: /tmp/pip-ephem-wheel-cache-atnd7tww/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/MyDrive/microsoft-learn-location-mention-recognition-challenge20240905-10153-193u9hv/Train_1.csv')
train.head(10)

,tweet_id,text,location
0,ID_1001136212718088192,NaN,EllicottCity
1,ID_1001136696589631488,"Flash floods struck a Maryland city on Sunday,...",Maryland
2,ID_1001136950345109504,State of emergency declared for Maryland flood...,Maryland
3,ID_1001137334056833024,Other parts of Maryland also saw significant d...,Baltimore Maryland
4,ID_1001138374923579392,"Catastrophic Flooding Slams Ellicott City, Mar...",Ellicott City Maryland
5,ID_1001138377717157888,WATCH: 1 missing after flash #FLOODING devasta...,Ellicott City Maryland
6,ID_1001139323075416064,NaN,Ellicott City Maryland
7,ID_1001139644023693312,NaN,NaN
8,ID_1001140017207459840,NaN,Maryland
9,ID_1001140276377935872,NaN,Maryland


In [6]:
test = pd.read_csv('/content/drive/MyDrive/microsoft-learn-location-mention-recognition-challenge20240905-10153-193u9hv/Test.csv')
test.head(10)

,tweet_id,text
0,ID_1001154804658286592,What is happening to the infrastructure in New...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...
5,ID_1001178904617476096,@CBSNews Our Harts gos out to a Fellow Soldier...
6,ID_1001179909245587456,CRAZY VIDEO. Roaring flash #floods struck a #M...
7,ID_1001180876548591616,I liked a @YouTube video BREAKING: Devastatin...
8,ID_1001182906130280448,Thank you to the first responders who are taki...
9,ID_1001185240256311296,CBS News: Ellicott City floods: Maryland offic...


In [7]:
# Display column names
print("Column Names Train:", train.columns)
print("Column Names Test:", test.columns)

Column Names Train: Index(['tweet_id', 'text', 'location'], dtype='object')
Column Names Test: Index(['tweet_id', 'text'], dtype='object')


In [8]:
# Basic info
print("Basic Info for train:\n", train.info())
print("Basic Info for test:\n", test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73072 entries, 0 to 73071
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  73072 non-null  object
 1   text      16448 non-null  object
 2   location  43460 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB
Basic Info for train:
 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2942 entries, 0 to 2941
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  2942 non-null   object
 1   text      2942 non-null   object
dtypes: object(2)
memory usage: 46.1+ KB
Basic Info for test:
 None


In [9]:
# Check for missing values
print("Missing Values in train:\n", train.isnull().sum())
print("Missing Values in test:\n", test.isnull().sum())

Missing Values in train:
 tweet_id        0
text        56624
location    29612
dtype: int64
Missing Values in test:
 tweet_id    0
text        0
dtype: int64


In [10]:
# Shape of data
print("Shape of train:", train.shape)
print("Shape of test:", test.shape)

Shape of train: (73072, 3)
Shape of test: (2942, 2)


In [11]:
# Basic statistics
print("Basic Statistics for train:\n", train.describe())
print("Basic Statistics for test:\n", test.describe())

Basic Statistics for train:
                      tweet_id  \
count                   73072   
unique                  73072   
top     ID_916205068281462784   
freq                        1   

                                                     text    location  
count                                               16448       43460  
unique                                              16448        7638  
top     #LadyGaga has donated $1M to help relief effor...  California  
freq                                                    1        4224  
Basic Statistics for test:
                      tweet_id  \
count                    2942   
unique                   2942   
top     ID_916099144116191232   
freq                        1   

                                                     text  
count                                                2942  
unique                                               2942  
top     RT @CBSSF: Carlos Santana Donates $100K To Mex...  
freq       

In [12]:
# Location Distribution
print("Location Distribution Train:\n", train['location'].value_counts())

Location Distribution Train:
 location
California                             4224
Kerala                                 2810
Puerto Rico                            1952
Florida                                1807
Mexico                                 1233
                                       ... 
Las Vegas Mandalay                        1
PUERTO RICAN                              1
Coatetelco Mexico Miacatlan Morelos       1
Queens Park                               1
Yabacoa                                   1
Name: count, Length: 7638, dtype: int64


In [13]:
train.shape

(73072, 3)

In [14]:
df_train = train.copy()

In [15]:
df_train.loc[4,'text']

'Catastrophic Flooding Slams Ellicott City, Maryland; Water Rescues Reported - The Weather Channel  via @GoogleNews'

In [16]:
df_test = test.copy()

In [17]:
df_train.dropna(inplace=True)

In [18]:
df_train.columns

Index(['tweet_id', 'text', 'location'], dtype='object')

In [19]:
df_train['location'].sample(10)

,location
1425,Athens Greece US
600,Baltimore- Maryland
64492,Mexico
37195,Pakistan
42888,Haiti
44727,Kaikoura
14999,Florence
37766,Ecuador
8068,Kerala Pinarayi Vijayan
65072,Mexico


In [20]:
# Filter for locations that are all uppercase
lowercase_locations = df_train[df_train['location'].str.islower()]['location']
print(len(lowercase_locations))
# get the locations as they appera in the original the dataframe
lowercase_locations.head(10)

239


,location
686,greece
857,greece
1116,greece
1118,greece
1292,greece
1301,pyrkagia
1356,greece
1368,greece
1369,mati
1544,greece


In [21]:
# Filter for locations that are all uppercase
uppercase_locations = df_train[df_train['location'].str.isupper()]['location']
print(len(uppercase_locations))
# get the locations as they appera in the original the dataframe
uppercase_locations.head(10)


386


,location
186,MD
320,MD
560,MARYLAND USA
602,DELAWARE MARYLAND PENNSYLVANIA VIRGINIA
603,DC DELAWARE MARYLAND VIRGINIA
838,ATHENS
918,EU
1461,US
1470,GREECE
1662,GREECE


### EDA


In [22]:
df_train['dataset'] = 'train'
df_test['dataset'] = 'test'
df_all = pd.concat([df_train, df_test], ignore_index=True)

print(df_all.shape)
df_all.sample(10)


(14791, 4)


,tweet_id,text,location,dataset
3378,ID_1108073443159953408,HELP NEEDEDὤFἿE Please help the survivors of #...,Zimbabwe,train
5386,ID_1176468046082428928,"RT @zburki: Mirpur, AJK, seems to have been mo...",AJK Mirpur,train
14685,ID_913250562740600832,Before and after VIIRS images of Puerto Rico s...,NaN,test
7423,ID_768889744952483840,RT @PerfectFdn: Italy Earthquake Relief and Re...,Italy,train
4225,ID_1111680669820817408,Delta residents have been busy collecting supp...,Nebraska,train
13022,ID_1167523469715746816,Five Red Cross volunteers from the Chicago are...,NaN,test
6051,ID_721733095708368896,RT @telesurenglish: Gracias Venezuela: Country...,Ecuador Venezuela,train
10780,ID_911666425298849792,RT @vquin96427: #HuracánMaría #HurricaneMaria ...,PuertoRico,train
14541,ID_911726497551065088,RT @KayaJones: My good friend & dancer Ana who...,NaN,test
9795,ID_907302304960978944,"It appears more people killed in Plano, Texas ...",Florida Texas,train


In [23]:
# Preprocess the text to remove or handle special characters (like '#')
def remove_hashtag(text):
    # For example, removing "#" from the text before tokenizing
    processed_text = re.sub(r'#', '', text)
    return processed_text

# preprocessed text column
df_all['processed_text'] = df_all['text'].apply(remove_hashtag)

In [24]:
def remove_emojis(text):
  """Removes emojis from a string.

  Args:
    text: The string to remove emojis from.

  Returns:
    The string with emojis removed.
  """
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r' ', text)


# Example usage with df
df_all['processed_text'] = df_all['processed_text'].apply(remove_emojis)


In [25]:
def remove_brackets(text):
  """Removes parentheses and square brackets from a given text."""
  processed_text = re.sub(r'[\[\]()]', '', text)
  return processed_text

# Apply the function to your 'processed_text' column
df_all['processed_text'] = df_all['processed_text'].apply(remove_brackets)

In [26]:
def remove_slash(text):
  """Replaces slashes with spaces in a given text."""
  processed_text = text.replace('/', ' ')
  return processed_text

# Apply the function to your 'processed_text' column
df_all['processed_text'] = df_all['processed_text'].apply(remove_slash)

In [27]:
df_all.sample(10)

,tweet_id,text,location,dataset,processed_text
11953,ID_1021796962851598336,"RT @ndtv: Unable to escape, families die in em...",NaN,test,"RT @ndtv: Unable to escape, families die in em..."
9316,ID_902652313751269376,Our thoughts are with Houston & plz text HARVE...,Houston,train,Our thoughts are with Houston & plz text HARVE...
8627,ID_798082918862028800,Powerful quake hits New Zealand’s South Island...,New Zealand,train,Powerful quake hits New Zealand’s South Island...
6389,ID_722042535074332672,"272 killed in powerful Ecuador quake, figure l...",Ecuador,train,"272 killed in powerful Ecuador quake, figure l..."
8178,ID_784698797251321856,Hurricane Matthew kills almost 900 in Haiti be...,Haiti,train,Hurricane Matthew kills almost 900 in Haiti be...
1769,ID_1037646675014180864,@CASA_India is responding to the urgent need o...,Kerala,train,@CASA_India is responding to the urgent need o...
10521,ID_910689695671001088,@ochocinco You should pay attention at whats g...,Mexico,train,@ochocinco You should pay attention at whats g...
10012,ID_908470787459502080,I activated the FL Small Business Emergency Br...,FL,train,I activated the FL Small Business Emergency Br...
12800,ID_1110555789633179648,A sister station of ours in Nebraska just help...,NaN,test,A sister station of ours in Nebraska just help...
6985,ID_730066926777536512,RT @RAN: Officials are surveying the damage of...,Alberta,train,RT @RAN: Officials are surveying the damage of...


In [28]:
df_train = df_all[df_all['dataset'] == 'train'].copy()
df_test = df_all[df_all['dataset'] == 'test'].copy()

df_train.drop('dataset', axis=1, inplace=True)
df_test.drop('dataset', axis=1, inplace=True)

print(df_train.shape)
print(df_test.shape)
print(df_train.sample(10))
print(df_test.sample(10))


(11849, 4)
(2942, 4)
                     tweet_id  \
4245   ID_1111711530624659456   
7223    ID_732730026710327296   
9944    ID_908175623138562048   
5351   ID_1168378509393313792   
5491   ID_1176486624777179136   
8716    ID_798468841311379456   
3866   ID_1110664766903717888   
3961   ID_1110919873482371072   
11610   ID_914124734559916032   
8813    ID_800820834029879296   

                                                    text           location  \
4245   Heres an awesome @AgDayTV story about one of o...           Nebraska   
7223   RT @LoisLeeTV: Two homes explode in Fort McMur...  Fort McMurray NYC   
9944   Stunning drone footage captures severe damage,...             Naples   
5351   The eye of hurricane Dorian 150mls north of Na...       Miami Nassau   
5491   RT @dawn_com: 2 dead, more than 100 injured as...           Pakistan   
8716   EU analysis suggests that initial & maybe even...           Kaikoura   
3866   Were ready for Friday - are you? Bid on the si...      

In [29]:
df_train['location'] = df_train['location'].apply(remove_hashtag)
df_train['location'] = df_train['location'].apply(remove_brackets)


## Preprocessing


In [30]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [67]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
#from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import pipeline


# Load the tokenizer and model
# Davlan/xlm-roberta-base-ner-hrl
# dslim/bert-base-NER-uncased
#Jean-Baptiste/roberta-large-ner-english
#FacebookAI/xlm-roberta-large-finetuned-conll03-english
#dslim/distilbert-NER


model_name = "dslim/bert-base-NER-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, device = 0) #device = 0


Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [68]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)  # Adjust test_size as needed

print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)


Training set shape: (9479, 4)
Validation set shape: (2370, 4)


In [69]:
# create a tokens column
train_df['tokens'] = train_df['processed_text'].apply(lambda x: tokenizer.tokenize(x))
for idx, row in train_df.head().iterrows():
    print(f"Text: {row['text']}")
    print("Processed Text:", row['processed_text'])
    print("Tokens:", row['tokens'])
    print("\n")

Text: #KeralaFloodRescue A small contribution from trivandrum, peroorkada Thalapathy vijay fans unit to flood affected peopleὤ2ὤ2
Processed Text: KeralaFloodRescue A small contribution from trivandrum, peroorkada Thalapathy vijay fans unit to flood affected peopleὤ2ὤ2
Tokens: ['kerala', '##fl', '##ood', '##res', '##cu', '##e', 'a', 'small', 'contribution', 'from', 'tri', '##van', '##drum', ',', 'per', '##oor', '##ka', '##da', 'tha', '##la', '##pathy', 'vijay', 'fans', 'unit', 'to', 'flood', 'affected', 'people', '##ω', '##2', '##ω', '##2']


Text: Peach County man transporting dozens of #dogs from #Texas to safety in #Tennessee after #Harvey damage to shelter.
Processed Text: Peach County man transporting dozens of dogs from Texas to safety in Tennessee after Harvey damage to shelter.
Tokens: ['peach', 'county', 'man', 'transporting', 'dozens', 'of', 'dogs', 'from', 'texas', 'to', 'safety', 'in', 'tennessee', 'after', 'harvey', 'damage', 'to', 'shelter', '.']


Text: Heavy #earthquake 

In [70]:
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

In [71]:
def tag_locations(text, locations, tokens):
    bio_tags = ['O'] * len(tokens)  # Default to 'O'

    # Loop through each location
    for loc in locations.split():  # split locations if multiple
        # Find all matches of the location in the text (case insensitive)
        matches = [(m.start(), m.end()) for m in re.finditer(re.escape(loc), text, re.IGNORECASE)]

        # For each match, find the corresponding tokens
        for start, end in matches:
            # Get the substring from the text that matches the location
            loc_substring = text[start:end]

            # Tokenize the location (to handle subwords)
            loc_tokens = tokenizer.tokenize(loc_substring)

            # Find where the location tokens appear in the full token list
            loc_token_indices = [i for i in range(len(tokens)) if ' '.join(tokens[i:i+len(loc_tokens)]) == ' '.join(loc_tokens)]

            # Apply BIO tags to the corresponding tokens
            if loc_token_indices:
                first_index = loc_token_indices[0]
                bio_tags[first_index] = 'B-LOC'  # First token gets B-LOC
                for i in range(1, len(loc_tokens)):
                    bio_tags[first_index + i] = 'I-LOC'  # Subsequent tokens get I-LOC

    return bio_tags

# Apply the function to the dataframe
train_df['tags'] = train_df.progress_apply(lambda row: tag_locations(row['processed_text'], row['location'], row['tokens']), axis=1)
for idx, row in train_df.head().iterrows():
    print("Text:", row['text'])
    print("Processed Text:", row['processed_text'])
    print("Tokens:", row['tokens'])
    print("Locations:", row['location'])
    print("Bio Tags:", row['tags'])
    print("\n")


  0%|          | 0/9479 [00:00<?, ?it/s]

Text: #KeralaFloodRescue A small contribution from trivandrum, peroorkada Thalapathy vijay fans unit to flood affected peopleὤ2ὤ2
Processed Text: KeralaFloodRescue A small contribution from trivandrum, peroorkada Thalapathy vijay fans unit to flood affected peopleὤ2ὤ2
Tokens: ['kerala', '##fl', '##ood', '##res', '##cu', '##e', 'a', 'small', 'contribution', 'from', 'tri', '##van', '##drum', ',', 'per', '##oor', '##ka', '##da', 'tha', '##la', '##pathy', 'vijay', 'fans', 'unit', 'to', 'flood', 'affected', 'people', '##ω', '##2', '##ω', '##2']
Locations: peroorkada trivandrum
Bio Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


Text: Peach County man transporting dozens of #dogs from #Texas to safety in #Tennessee after #Harvey damage to shelter.
Processed Text: Peach County man transporting dozens of dogs from Texas to safety in Tennessee afte

In [72]:
def extract_locations(tokens, tags, tokenizer):
    locations = []
    current_location = []

    # Loop over tokens and tags together
    for token, tag in zip(tokens, tags):
        # If tag is B-LOC, start a new location
        if tag == 'B-LOC':
            # If we have an ongoing location, add it to the list
            if current_location:
                locations.append(current_location)  # Add list of tokens for the current location
                current_location = []
            current_location.append(token)
        # If tag is I-LOC, continue the current location
        elif tag == 'I-LOC' and current_location:
            current_location.append(token)
        # If tag is not B-LOC or I-LOC and we have an ongoing location, end it
        else:
            if current_location:
                locations.append(current_location)  # Add list of tokens for the current location
                current_location = []

    # Catch any remaining location at the end
    if current_location:
        locations.append(current_location)  # Add list of tokens for the last location

    # Decode the tokens back to their original form
    decoded_locations = [tokenizer.convert_tokens_to_string(loc) for loc in locations]

    # Sort the decoded locations alphabetically
    decoded_locations.sort()

    # Join sorted locations with a comma separator
    formatted_locations = ", ".join(decoded_locations)

    return formatted_locations


In [73]:
# Apply the function to the dataframe
train_df['extracted_locations'] = train_df.progress_apply(lambda row: extract_locations(row['tokens'], row['tags'], tokenizer), axis=1)

# View the updated dataframe with extracted and sorted locations
print(train_df[['location', 'extracted_locations']])

  0%|          | 0/9479 [00:00<?, ?it/s]

                           location              extracted_locations
1199          peroorkada trivandrum           peroorkada, trivandrum
9557   Peach County Tennessee Texas  county, peach, tennessee, texas
5605                      Ak Mirpur                       ak, mirpur
6584           London San Francisco           francisco, london, san
5268                        Bahamas                          bahamas
...                             ...                              ...
11284                   Puerto Rico                     puerto, rico
5191                        Bahamas                          bahamas
5390         Mangla SDMA AJK mirpur        ajk, mangla, mirpur, sdma
860                          Greece                           greece
7270                  Fort McMurray                   fort, mcmurray

[9479 rows x 2 columns]


In [74]:
# Find rows where 'extracted_locations' is an empty list
empty_extracted_locations = train_df[train_df['extracted_locations'] == '']

# Print the number of rows with empty 'extracted_locations'
print("Number of rows with empty 'extracted_locations':", len(empty_extracted_locations))

# If you want to see these rows:
empty_extracted_locations


Number of rows with empty 'extracted_locations': 0


,tweet_id,text,location,processed_text,tokens,tags,extracted_locations


# Training

In [75]:
labels = model.config.id2label
print(labels)

{0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}


In [76]:
max_len = 0
for _, row in train_df.iterrows():
  max_len = max(max_len, len(row['tokens']))

print(f"Max token length: {max_len}")


Max token length: 362


In [77]:
import torch

# Define a maximum length for padding/truncating
max_len = 362

# Helper function to prepare input IDs, attention masks, and labels
def prepare_inputs_and_labels(tokens, bio_tags, max_len, tokenizer):
    # Convert tokens to input IDs
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Truncate sequences longer than max_len
    input_ids = input_ids[:max_len]
    bio_tags = bio_tags[:max_len]

    # Create attention masks (1 for actual tokens, 0 for padding)
    attention_mask = [1] * len(input_ids)

    # Pad sequences to max_len
    input_ids += [tokenizer.pad_token_id] * (max_len - len(input_ids))
    attention_mask += [0] * (max_len - len(attention_mask))
    bio_tags += ['O'] * (max_len - len(bio_tags))  # Pad labels with 'O'

    # Convert BIO tags to numeric labels using your label mapping
    label_map = {'O': 0, 'B-MISC': 1, 'I-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-LOC': 7, 'I-LOC': 8}
    labels = [label_map[tag] for tag in bio_tags]

    return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(labels)

# Example on how to apply this to the dataset
train_inputs = []
train_masks = []
train_labels = []

for _, row in train_df.iterrows():
    tokens = row['tokens']
    bio_tags = row['tags']

    input_ids, attention_mask, labels = prepare_inputs_and_labels(tokens, bio_tags, max_len, tokenizer)

    train_inputs.append(input_ids)
    train_masks.append(attention_mask)
    train_labels.append(labels)

# Convert lists to tensors
train_inputs = torch.stack(train_inputs)
train_masks = torch.stack(train_masks)
train_labels = torch.stack(train_labels)

print("Sample input IDs:", train_inputs[0])
print("Sample attention mask:", train_masks[0])
print("Sample labels:", train_labels[0])


Sample input IDs: tensor([ 8935, 10258, 17139,  6072, 10841,  2063,  1037,  2235,  6691,  2013,
        13012,  6212, 21884,  1010,  2566, 16506,  2912,  2850, 22794,  2721,
        20166, 17027,  4599,  3131,  2000,  7186,  5360,  2111, 29739,  2475,
        29739,  2475,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

In [78]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create TensorDataset
train_data = TensorDataset(train_inputs, train_masks, train_labels)

# Create DataLoader with random sampling
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Verify data loading by checking a batch
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print("Batch input IDs:", input_ids)
    print("Batch attention masks:", attention_mask)
    print("Batch labels:", labels)
    break  # print the first batch for now


Batch input IDs: tensor([[10958, 21886, 12338,  ...,     0,     0,     0],
        [ 2065, 26060,  2064,  ...,     0,     0,     0],
        [ 4550,  2039,  3169,  ...,     0,     0,     0],
        ...,
        [ 2057,  2439,  2256,  ...,     0,     0,     0],
        [19387,  1030,  1996,  ...,     0,     0,     0],
        [19387,  1030,  3149,  ...,     0,     0,     0]])
Batch attention masks: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Batch labels: tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [79]:
print(train_inputs.shape)  # Should match train_labels and train_masks
print(train_masks.shape)
print(train_labels.shape)

torch.Size([9479, 362])
torch.Size([9479, 362])
torch.Size([9479, 362])


In [80]:
from datasets import Dataset
from transformers import DataCollatorForTokenClassification

# Convert your dataset into a Hugging Face Dataset
train_data_dict = {
    'input_ids': train_inputs,
    'attention_mask': train_masks,
    'labels': train_labels
}

train_dataset = Dataset.from_dict(train_data_dict)

# Create a data collator for dynamic padding (so it pads to the largest sequence in a batch)
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)


In [81]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="steps",          # Evaluate every specified number of steps
    eval_steps=500,                       # Evaluate every 500 steps
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)


In [82]:
import numpy as np
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

# Define label names for evaluation
label_names = ['O', 'B-MISC', 'I-MISC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (we use -100 in Hugging Face for ignored labels)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    # Calculate metrics
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)

    # print the classification report
    print(classification_report(true_labels, true_predictions))

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [83]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [84]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.005800,0.003636,0.998780,0.860452,0.956302,0.905849
1000,0.003800,0.002035,0.999328,0.941000,0.940531,0.940766
1500,0.001600,0.001364,0.999527,0.943931,0.973881,0.958672
2000,0.001400,0.000929,0.999698,0.965487,0.976562,0.970993
2500,0.001200,0.000599,0.999802,0.980695,0.981673,0.981184
3000,0.001000,0.000376,0.999878,0.983560,0.992021,0.987772
3500,0.000400,0.000272,0.999917,0.987265,0.995512,0.991371
4000,0.000500,0.000192,0.999943,0.993707,0.994140,0.993924
4500,0.000300,0.000124,0.999966,0.995766,0.997008,0.996387
5000,0.000200,0.000087,0.999977,0.996516,0.998442,0.997478


              precision    recall  f1-score   support

         LOC       0.86      0.96      0.91     16042

   micro avg       0.86      0.96      0.91     16042
   macro avg       0.86      0.96      0.91     16042
weighted avg       0.86      0.96      0.91     16042

              precision    recall  f1-score   support

         LOC       0.94      0.94      0.94     16042

   micro avg       0.94      0.94      0.94     16042
   macro avg       0.94      0.94      0.94     16042
weighted avg       0.94      0.94      0.94     16042

              precision    recall  f1-score   support

         LOC       0.94      0.97      0.96     16042

   micro avg       0.94      0.97      0.96     16042
   macro avg       0.94      0.97      0.96     16042
weighted avg       0.94      0.97      0.96     16042

              precision    recall  f1-score   support

         LOC       0.97      0.98      0.97     16042

   micro avg       0.97      0.98      0.97     16042
   macro avg     

TrainOutput(global_step=5930, training_loss=0.0016861172462263872, metrics={'train_runtime': 8964.4662, 'train_samples_per_second': 10.574, 'train_steps_per_second': 0.662, 'total_flos': 1.751308421961348e+16, 'train_loss': 0.0016861172462263872, 'epoch': 10.0})

In [85]:
trainer.save_model("./model")

In [86]:
val_df

,tweet_id,text,location,processed_text
8192,ID_797790014872485888,I cant believe people are asking for specifics...,NZ nz,I cant believe people are asking for specifics...
900,ID_1024023673294991360,Great piece by @maledictus on how Greeces migr...,Europe,Great piece by @maledictus on how Greeces migr...
3898,ID_1110760931225714688,Donate to support this teacher from my daughte...,Nebraska,Donate to support this teacher from my daughte...
9794,ID_907298890206121984,Florida Highway Patrol says roads to the Flori...,Florida,Florida Highway Patrol says roads to the Flori...
2820,ID_1064146031028420608,Good Morning Patriots!!! My prayers ὤFἿB this ...,California,Good Morning Patriots!!! My prayers ὤFἿB this ...
...,...,...,...,...
7926,ID_783772478820581376,RT @water_mission: Help make #HurricaneMatthew...,Haiti,RT @water_mission: Help make HurricaneMatthew ...
8426,ID_797879012240392192,Civil Defence has issued a tsunami warning fro...,Dunedin Napier,Civil Defence has issued a tsunami warning fro...
4387,ID_1112425377576689664,Consider a donation to this Nebraska rescue in...,Nebraska,Consider a donation to this Nebraska rescue in...
11558,ID_913874387505426432,Help us support Mexico at this Sundays concert...,Mexico,Help us support Mexico at this Sundays concert...


In [87]:
!pwd

/content


In [88]:
!ls

drive  model  results  sample_data


In [89]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the trained model and tokenizer
loaded_model = AutoModelForTokenClassification.from_pretrained("/content/model")
loaded_tokenizer = AutoTokenizer.from_pretrained(model_name)  # Use the same tokenizer you used for training


In [90]:
# Move the model to the GPU
loaded_model.to('cuda')

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [91]:
def predict_locations(text, loaded_model, loaded_tokenizer):
    # Tokenize the text
    tokens = loaded_tokenizer.tokenize(text)

    # Convert tokens to input IDs and create attention mask
    input_ids = loaded_tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(input_ids)

    # Pad to the same length as used during training
    input_ids = input_ids + [loaded_tokenizer.pad_token_id] * (max_len - len(input_ids))
    attention_mask = attention_mask + [0] * (max_len - len(attention_mask))

    # Convert to PyTorch tensors
    input_ids = torch.tensor([input_ids]).to('cuda')
    attention_mask = torch.tensor([attention_mask]).to('cuda')

    # Make prediction
    with torch.no_grad():
        outputs = loaded_model(input_ids, attention_mask=attention_mask)

    # Get the predicted labels
    predicted_label_ids = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy().tolist()

    # Convert label IDs to label names
    predicted_labels = [label_names[label_id] for label_id in predicted_label_ids]

    # Extract locations from tokens and predicted labels
    locations = []
    current_location = []
    for token, label in zip(tokens, predicted_labels):
        if label == 'B-LOC':
            if current_location:
                locations.append(' '.join(current_location))
                current_location = []
            current_location.append(token)
        elif label == 'I-LOC':
            current_location.append(token)
        else:
            if current_location:
                locations.append(' '.join(current_location))
                current_location = []

    if current_location:
        locations.append(' '.join(current_location))

    # Decode the extracted locations to their original form
    decoded_locations = []
    for loc in locations:
        # Use the tokenizer's decode method instead
        decoded_location = loaded_tokenizer.decode(loaded_tokenizer.convert_tokens_to_ids(loc.split()), skip_special_tokens=True)
        decoded_locations.append(decoded_location)

    # Sort the decoded locations alphabetically
    #decoded_locations = sorted(set(decoded_locations))

    return decoded_locations



In [101]:
val_df['predicted_locations'] = val_df['processed_text'].apply(lambda text: predict_locations(text, loaded_model, loaded_tokenizer))
print(val_df[['text', 'location', 'predicted_locations']])


                                                    text  \
8192   I cant believe people are asking for specifics...   
900    Great piece by @maledictus on how Greeces migr...   
3898   Donate to support this teacher from my daughte...   
9794   Florida Highway Patrol says roads to the Flori...   
2820   Good Morning Patriots!!! My prayers ὤFἿB this ...   
...                                                  ...   
7926   RT @water_mission: Help make #HurricaneMatthew...   
8426   Civil Defence has issued a tsunami warning fro...   
4387   Consider a donation to this Nebraska rescue in...   
11558  Help us support Mexico at this Sundays concert...   
4536   RT @Swan_Enterprise: Flood relief headed from ...   

                     location  \
8192                    NZ nz   
900                    Europe   
3898                 Nebraska   
9794                  Florida   
2820               California   
...                       ...   
7926                    Haiti   
8426           

In [102]:
val_df[['location', 'predicted_locations']].sample(50)

,location,predicted_locations
9669,Caribbean StMartin,[stmartin]
7007,Fort McMurray,"[fort, mcmurray]"
3305,Mozambique,[mozambique]
5361,Nassau,[nassau]
8330,Canterbury Rotherham,"[rotherham, canterbury]"
3692,Nebraska,[nebraska]
9558,Louisiana,[louisiana]
8394,Kai New Zealand,"[new, zealand, kai]"
3021,California,[california]
7958,Haiti,[haiti]


In [103]:
val_df.to_csv('val_df.csv', index=False)


In [104]:
#!ls

In [105]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename='val_df.csv')

In [106]:
df_test

,tweet_id,text,location,processed_text,predicted_locations
11849,ID_1001154804658286592,What is happening to the infrastructure in New...,NaN,What is happening to the infrastructure in New...,"[new, england, orleans, maryland]"
11850,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...,NaN,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...,"[ellicott, city, maryland]"
11851,ID_1001155756371136512,RT @TIME: Police searching for missing person ...,NaN,RT @TIME: Police searching for missing person ...,"[ellicott, city, maryland]"
11852,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...,NaN,Flash Flood Tears Through Maryland Town For Se...,"[maryland, ellicott, city, md.]"
11853,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...,NaN,Ellicott City FLOODING Pictures: Maryland Gove...,"[ellicott, city, maryland]"
...,...,...,...,...,...
14786,ID_915017703055749120,@PurinaCatChow he wants to donate some food to...,NaN,@PurinaCatChow he wants to donate some food to...,[mexico]
14787,ID_915026957758328832,@HannahStocking I live the Mexico earthquake a...,NaN,@HannahStocking I live the Mexico earthquake a...,"[mexico, las, vegas]"
14788,ID_915253441726889984,RT @GlobalCalgary: WATCH: National Taco Day in...,NaN,RT @GlobalCalgary: WATCH: National Taco Day in...,"[calgary, mexico, city]"
14789,ID_915971980859400192,Oaxaca & Chiapas – Mexico’s poorest states – h...,NaN,Oaxaca & Chiapas – Mexico’s poorest states – h...,"[oaxaca, chiapas, mexico]"


In [107]:
df_test['predicted_locations'] = df_test['processed_text'].apply(lambda text: predict_locations(text, loaded_model, loaded_tokenizer))
df_test[['text', 'predicted_locations']].sample(50)


,text,predicted_locations
13865,"RT @RT_com: Over 870 killed, 350,000 in need o...",[haiti]
14052,"Cyclone Mora: Snapped power lines, poor networ...",[srilanka]
13315,"RT @WTOP: Ecuador hit by powerful earthquake, ...",[ecuador]
13321,"@MattAboutTown listen Matty, dear, people have...","[ecuador, australia]"
14541,RT @KayaJones: My good friend & dancer Ana who...,[puerto]
13615,More Evacuations From Fort McMurray Wildfire ...,"[fort, mcmurray, saskatoon]"
13727,i really do hope the hurricane doesnt do TOO m...,[haiti]
14729,#PuertoRico dies while #DonaldTrump flies. #Sa...,[puertorico]
14508,Welcome To Naija Cool News: Mexico earthquake:...,[mexico]
13223,Catastrophe hits #Pakistan again - earthquake ...,"[pakistan, azad, kashmir, mirpur]"


In [108]:
# Find rows where 'predicted_locations' is an empty list
empty_predicted_locations = df_test[df_test['predicted_locations'].apply(lambda x: len(x) == 0)]

# Print the number of rows with empty 'predicted_locations'
print("Number of rows with empty 'predicted_locations':", len(empty_predicted_locations))

# If you want to see these rows:
empty_predicted_locations


Number of rows with empty 'predicted_locations': 2


,tweet_id,text,location,processed_text,predicted_locations
13970,ID_798002317127655424,Children-unexpectedly-home-today-due-to-earthq...,NaN,Children-unexpectedly-home-today-due-to-earthq...,[]
14034,ID_870198877101727744,1st part of #FloodSL Relief FoodAID collection...,NaN,1st part of FloodSL Relief FoodAID collection ...,[]


In [109]:
df_test.loc[13970,'text']

'Children-unexpectedly-home-today-due-to-earthquake damage #eqnz #earthquake #wellingtonnz'

In [110]:
df_test.loc[13970,'processed_text']

'Children-unexpectedly-home-today-due-to-earthquake damage eqnz earthquake wellingtonnz'

In [111]:
df_test.loc[14034,'text']

'1st part of #FloodSL Relief FoodAID collection over successfully Dear international Organizations Please support for US to help the people'

In [112]:
df_test[['text', 'predicted_locations']].sample(10)

,text,predicted_locations
12736,Casa C 295 H avion civil fantástico ! Thanks r...,"[chimoio, beira]"
14704,My @VogueMexico cover comes out today! Proceed...,[mexico]
14456,RT-Please we need your help!!ὤFἿC #mexico #hel...,[mexico]
12860,This relief effort needs to get more attention...,"[south, dakota, nebraska]"
12812,@lutzviewfarms is working on getting a load of...,[nebraska]
14094,Sri Lanka : Flood and Landslide Emergency Resp...,"[sri, lanka]"
14086,US$4 million @UNCERF grant allocated to provid...,[srilanka]
12099,Modi sanctions Rs 500 crore for flood hit Kera...,[kerala]
13723,Hurricane Matthew batters Haiti as US authorit...,"[haiti, us]"
14357,"675,000+ in Florida & 5,638 with Clay Electric...",[florida]


In [113]:
df_test.to_csv('df_test.csv', index=False)


In [114]:
create_download_link(filename='df_test.csv')

In [115]:
df_test.columns

Index(['tweet_id', 'text', 'location', 'processed_text',
       'predicted_locations'],
      dtype='object')

In [116]:
df_test = df_test.rename(columns={'tweet_id': 'ID', 'predicted_locations': 'Locations'})


In [117]:
submission_df = df_test[['ID', 'Locations']]
submission_df.to_csv('submission.csv', index=False)


In [118]:
submission_df

,ID,Locations
11849,ID_1001154804658286592,"[new, england, orleans]"
11850,ID_1001155505459486720,"[ellicott, city, maryland]"
11851,ID_1001155756371136512,"[ellicott, city, maryland]"
11852,ID_1001159445194399744,"[maryland, ellicott, city, md.]"
11853,ID_1001164907587538944,"[ellicott, city, maryland]"
...,...,...
14786,ID_915017703055749120,[mexico]
14787,ID_915026957758328832,"[mexico, las, vegas]"
14788,ID_915253441726889984,"[calgary, mexico, city]"
14789,ID_915971980859400192,"[oaxaca, chiapas, mexico]"
